<a href="https://colab.research.google.com/github/aditya26091999/AssociativeClassifier/blob/master/AssociativeClassifier_SV_PHD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Associative Classifier (Classification based on Apriori Algorithm generated rules)

##### Pre-requisite setup (Data cleaning, Transacional dataset preparation)

In [ ]:
#Importing necessary dependencies and libraries
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
#Upload the temporary dataset csv file to Google Colab Files section
#Update the below method to reference the appropriate CSV File.
original_dataset = pd.read_csv('/BuyComputerDataset.csv')
print('------------------\n#DATASET VALUES\n------------------')
display(original_dataset)

In [ ]:
print('------------------\n#DATASET INFO()\n------------------')
original_dataset.info()
print('\n------------------\n#DATASET DESCRIBE()\n------------------')
original_dataset.describe()

**Coverting the dataset into transactional dataset**

In [ ]:
#Create a independent copy of the original dataset to form transactional_dataset
transactional_dataset = original_dataset.copy(deep=True)

#Create a transaction_symbol reference dataset
transactional_symbol_reference = pd.DataFrame(columns=['Attribute', 'Value', 'Symbol'])

#List to track non-categorical columns
non_categorical_columns = list()

#Traverse the dataset columnwise to encode columnar data into symbols for the transactional_dataset
#Skipping the first column='RID'
print('------------------\n#CONVERTING TO TRANSACTIONAL DATASET\n------------------')
for attribute in transactional_dataset.columns:
  attribute_dtype = transactional_dataset[attribute].dtype

  if transactional_dataset[attribute].dtype == 'object':
    unique_attribute_values = list(transactional_dataset[attribute].unique())
    print('\nAttribute : ' + attribute + ' is categorical with unique value set : ', unique_attribute_values)

    #Transactional data encoding starts for that attribute column values
    for index, unique_val in enumerate(unique_attribute_values):
      transaction_symbol = attribute[:2] + '_' +str(index)
      transactional_symbol_reference = transactional_symbol_reference.append({'Attribute' : attribute, 'Value' : unique_val, 'Symbol' : transaction_symbol}, ignore_index = True)
      transactional_dataset.loc[transactional_dataset[attribute] == unique_val, attribute] = transaction_symbol

  else:
    non_categorical_columns.append([attribute,attribute_dtype])

print("\n------------------\n#CURRENT TRANSACTIONAL DATASET LOOKS LIKE THIS\n------------------")
display(transactional_dataset)

print("\n------------------\n#COLUMNS THAT COULDN'T BE CONVERTED\n------------------")
for entry in non_categorical_columns:
  print('\nAttribute : ' + entry[0] + ' is of type : ', entry[1])

print("\n------------------\n#WRITE CUSTOM LOGIC TO CONVERT THEM INTO CATEGORICAL\n------------------")

**Custom Logic for converting Non-Categorical Columns into Categorical**

1. For Age Attribute

In [ ]:
#Custom Logic to convert Age Column into Categorical Values

#Defining the Category labels for Age Column and adding entries to 'transactional_symbol_reference' dataset
age_attribute_categories = {
    "AttributeName":"Age",
    "CategoryValues" : ["<=30","31...40",">40"]
}

#Automatic generation of transaction symbols for 'transaction_symbol_reference' dataset
attribute = age_attribute_categories['AttributeName']
for index, unique_val in enumerate(age_attribute_categories['CategoryValues']):
  transaction_symbol = attribute[:2] + '_' +str(index)
  transactional_symbol_reference = transactional_symbol_reference.append({'Attribute' : attribute, 'Value' : unique_val, 'Symbol' : transaction_symbol}, ignore_index = True)

#Writing the rules for encoding the non-categorical values
a = list(transactional_dataset.query('Age <= 30').index)
b = list(transactional_dataset.query('Age >= 31 and Age<=40').index)
c = list(transactional_dataset.query('Age > 40').index)


#Encoding the 'transactional_dataset' - Age attribute values with transaction symbols
transactional_dataset.loc[a, 'Age'] = 'Ag_0'
transactional_dataset.loc[b, 'Age'] = 'Ag_1'
transactional_dataset.loc[c, 'Age'] = 'Ag_2'

print("\n------------------\n#CURRENT TRANSACTIONAL DATASET LOOKS LIKE THIS\n------------------")
display(transactional_dataset)

**Pre-requisites for AssociativeClassifier Completed**

**Showing the completed assets**

In [ ]:
print("\n------------------\n#TRANSACTIONAL DATASET LOOKS LIKE THIS\n------------------")
display(transactional_dataset)


print("\n------------------\n#TRANSACTIONAL SYMBOL REFERENCE TABLE LOOKS LIKE THIS\n------------------")
display(transactional_symbol_reference)

##### Apriori Algorithm (Generating Association Rule mining rules)